## Exercice 1

##### Réponse / idées : 

In [41]:
!pip install unidecode

In [1]:
import matplotlib as plt
import json 
import urllib
import pandas as pd
import unidecode
from sklearn.metrics import precision_score, classification_report, confusion_matrix, roc_curve, roc_auc_score, balanced_accuracy_score, ConfusionMatrixDisplay

In [3]:
#Sentence conversion
sentence = "Je recherche un restaurant à Gruissan"
sentence = unidecode.unidecode(sentence)
sentence = sentence.replace(" ", "%20")
response = urllib.request.urlopen('http://192.168.99.100:7777/api/intent?sentence=' + 'trouve%20a%20champigny-sur-marne%20ou%20manger%20de%20la%20cuisine%20tcheque')
data = json.load(response)   
print(data)

{'find-train': 0.013913900591433048, 'irrelevant': 0.15839166939258575, 'find-flight': 0.00852132961153984, 'find-restaurant': 0.701693594455719, 'purchase': 0.0683024451136589, 'find-around-me': 0.029640382155776024, 'provide-showtimes': 0.00977227371186018, 'find-hotel': 0.009764429181814194}


In [4]:
#Reading training_set file 
with open('training_set.json', encoding="utf8") as json_file:
    training_data = json.load(json_file)
    
#Reading testing_set file 
with open('testing_set.json', encoding="utf8") as json_file:
    testing_data = json.load(json_file)

In [5]:
#Counting the number of data (train) for each intent
train_data = {'find-train': 0, 'irrelevant': 0, 'find-flight': 0, 'find-restaurant': 0,
           'purchase': 0, 'find-around-me' : 0, 'provide-showtimes' : 0, 'find-hotel' : 0}
for i in range(len(training_data)):
    for intent, count in train_data.items():
        if training_data[i]['intent'] == intent :
            count = count + 1
            train_data[intent] = count

#Counting each intent iteration
train_data

{'find-train': 143,
 'irrelevant': 3852,
 'find-flight': 142,
 'find-restaurant': 469,
 'purchase': 613,
 'find-around-me': 383,
 'provide-showtimes': 117,
 'find-hotel': 316}

In [6]:
#Counting the number of data (test) for each intent
test_data = {'find-train': 0, 'irrelevant': 0, 'find-flight': 0, 'find-restaurant': 0,
           'purchase': 0, 'find-around-me' : 0, 'provide-showtimes' : 0, 'find-hotel' : 0}
for i in range(len(testing_data)):
    for intent, count in test_data.items():
        if testing_data[i]['intent'] == intent :
            count = count + 1
            test_data[intent] = count

#Counting each intent iteration
test_data

{'find-train': 21,
 'irrelevant': 677,
 'find-flight': 24,
 'find-restaurant': 93,
 'purchase': 114,
 'find-around-me': 67,
 'provide-showtimes': 14,
 'find-hotel': 55}

In [7]:
#Put training and testing data into a DataFrame
df_training = pd.DataFrame(training_data)
df_testing = pd.DataFrame(testing_data)

In [8]:
#Summary
df_testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1065 entries, 0 to 1064
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   intent    1065 non-null   object
 1   sentence  1065 non-null   object
dtypes: object(2)
memory usage: 16.8+ KB


In [9]:
#10 random sample
df_training.sample(n=10)

,intent,sentence
3510,irrelevant,Devrais-je me maquiller ?
1784,find-restaurant,J'ai comme budget 25€/p suggere moi un endroit...
5395,irrelevant,je veux avoir des alarmes pour tout mes rendez...
2166,purchase,Je voudrais précommander une console
1152,find-flight,"Holà senor, je suis à la recherche de vols pas..."
5252,find-flight,"Coucou Génie, on m’a offert deux places pour l..."
3081,irrelevant,nrj 12 programmation
1208,find-hotel,J'aimerai trouver un lieu où il est possible d...
3502,find-restaurant,Je voudrais dîner dimanche dans un restau haut...
3364,irrelevant,bonjour oui dit


In [10]:
df_training.describe()

,intent,sentence
count,6035,6035
unique,8,6035
top,irrelevant,Comment gagner de l'argent facilement sur inte...
freq,3852,1


In [11]:
#Count all values for a feature in training set
df_training["intent"].value_counts()

irrelevant           3852
purchase              613
find-restaurant       469
find-around-me        383
find-hotel            316
find-train            143
find-flight           142
provide-showtimes     117
Name: intent, dtype: int64

In [12]:
#Count all values for a feature in testing set
df_testing["intent"].value_counts()

irrelevant           677
purchase             114
find-restaurant       93
find-around-me        67
find-hotel            55
find-flight           24
find-train            21
provide-showtimes     14
Name: intent, dtype: int64

In [13]:
#First 10 intents of the testing set
df_testing["intent"].head(n=10)

0         irrelevant
1         irrelevant
2         irrelevant
3           purchase
4    find-restaurant
5         irrelevant
6         find-train
7         irrelevant
8         irrelevant
9         irrelevant
Name: intent, dtype: object

In [16]:
train_data = {'find-train': 0, 'irrelevant': 0, 'find-flight': 0, 'find-restaurant': 0,
           'purchase': 0, 'find-around-me' : 0, 'provide-showtimes' : 0, 'find-hotel' : 0}

In [21]:
#Get the model prediction for the testing set
#urllib.parse.quote
intents = []
probabilities = []
scores = []
for i in range(len(testing_data)):
    sentence = unidecode.unidecode(testing_data[i]['sentence'])
    sentence = sentence.replace(" ", "%20")
    response = urllib.request.urlopen('http://192.168.99.100:7777/api/intent?sentence='+sentence)
    datas = json.load(response)
    (intent, probability) = ("irrelevant", 0)
    score = []
    for data in datas:
        score.append(datas[data])
        if(datas[data] > probability):
            probability = datas[data]
            intent = data
    intents.append(intent)
    probabilities.append(probability)
    scores.append(score)
        

In [22]:
df_pred["intent"] = pd.DataFrame(intents)
df_pred["probability"] = pd.DataFrame(probabilities)

NameError: name 'df_pred' is not defined

In [195]:
#First 10 intents
df_pred["intent"].head(n=10)

0         irrelevant
1         irrelevant
2         irrelevant
3         irrelevant
4    find-restaurant
5         irrelevant
6         find-train
7         irrelevant
8         irrelevant
9         irrelevant
Name: intent, dtype: object

In [ ]:
test_data = {'find-train': 0, 'irrelevant': 0, 'find-flight': 0, 'find-restaurant': 0,
           'purchase': 0, 'find-around-me' : 0, 'provide-showtimes' : 0, 'find-hotel' : 0}
for i in range(df_pred["intent"].shape[0]):
    for intent, count in test_data.items():
        if testing_data[i]['intent'] == intent :
            count = count + 1
            test_data[intent] = count

In [178]:
def plot_cm(y_true, y_pred, cmap, xticks_rotation, normalize):
    cm = confusion_matrix(y_true, y_pred, normalize = normalize)
    disp = ConfusionMatrixDisplay(confusion_matrix = cm)
    disp.plot(cmap, ax=None, xticks_rotation = 'horizontal')

In [148]:
#First 10 scores
print(scores)

[[0.012382732704281807, 0.41360554099082947, 0.02624133974313736, 0.06420969218015671, 0.35677096247673035, 0.09145810455083847, 0.004994788207113743, 0.03033687174320221], [0.03319886699318886, 0.543869137763977, 0.025731701403856277, 0.02996217831969261, 0.08921506255865097, 0.06310330331325531, 0.034597016870975494, 0.18032273650169373], [0.017024269327521324, 0.7049711346626282, 0.020258499309420586, 0.10295719653367996, 0.0574716255068779, 0.03430013358592987, 0.025665007531642914, 0.037352126091718674], [0.011617439799010754, 0.42591533064842224, 0.015384767204523087, 0.025107793509960175, 0.3575160801410675, 0.09943437576293945, 0.013050340116024017, 0.05197380110621452], [0.013913900591433048, 0.15839166939258575, 0.00852132961153984, 0.701693594455719, 0.0683024451136589, 0.029640382155776024, 0.00977227371186018, 0.009764429181814194], [0.060533374547958374, 0.521228015422821, 0.05057314783334732, 0.07879552990198135, 0.07477863878011703, 0.07833261042833328, 0.07157700508832

In [181]:
labels = ['find-around-me', 'find-flight', 'find-hotel', 'find-restaurant', 'find-train', 'irrelevant', 'provide-showtimes', 'purchase'] 
print(confusion_matrix(df_testing["intent"], df_pred["intent"], labels = labels, normalize = "true"))
#plot_cm(df_testing["intent"], df_pred["intent"], 'viridis', 'horizontal', 'true')

[[0.40298507 0.         0.         0.02985075 0.         0.49253731
  0.         0.07462687]
 [0.         0.25       0.         0.         0.         0.625
  0.04166667 0.08333333]
 [0.01818182 0.         0.49090909 0.01818182 0.         0.47272727
  0.         0.        ]
 [0.         0.         0.02150538 0.61290323 0.         0.35483871
  0.         0.01075269]
 [0.         0.         0.         0.         0.66666667 0.28571429
  0.         0.04761905]
 [0.         0.         0.00738552 0.         0.0014771  0.98227474
  0.         0.00886263]
 [0.         0.         0.         0.         0.         0.71428571
  0.28571429 0.        ]
 [0.         0.00877193 0.         0.         0.         0.40350877
  0.         0.5877193 ]]


In [183]:
print(classification_report(df_testing["intent"], df_pred["intent"], digits=2))

                   precision    recall  f1-score   support

   find-around-me       0.96      0.40      0.57        67
      find-flight       0.86      0.25      0.39        24
       find-hotel       0.79      0.49      0.61        55
  find-restaurant       0.95      0.61      0.75        93
       find-train       0.93      0.67      0.78        21
       irrelevant       0.80      0.98      0.88       677
provide-showtimes       0.80      0.29      0.42        14
         purchase       0.82      0.59      0.68       114

         accuracy                           0.81      1065
        macro avg       0.86      0.53      0.63      1065
     weighted avg       0.83      0.81      0.79      1065



In [184]:
print(roc_auc_score(df_testing["intent"], scores, multi_class='ovo'))

0.5724183413000686


In [155]:
#Compute the balanced accuracy
#It is defined as the average of recall obtained on each class.
#The best value is 1 and the worst value is 0

print(balanced_accuracy_score(df_testing["intent"], df_pred["intent"]))

0.5348965479344527


## Exercice 2

##### Réponse / idées : 

In [ ]:
labels = ['find-around-me', 'find-flight', 'find-hotel', 'find-restaurant', 'find-train', 'irrelevant', 'provide-showtimes', 'purchase'] 

mean_std = pd.DataFrame()

## Exercice 3

##### Réponse / idées : 

## Exercice 4

##### Réponse / idées : 

## Exercice 5

##### Réponse / idées : 